# Parte 4: Cuantificación

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf

tf.random.set_seed(seed)
import os

os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']

## Obtener el conjunto de datos de etiquetado de jets de Open ML

In [ ]:
X_train_val = np.load('X_train_val.npy')
X_test = np.load('X_test.npy')
y_train_val = np.load('y_train_val.npy')
y_test = np.load('y_test.npy')
classes = np.load('classes.npy', allow_pickle=True)

## Construir un modelo
Esta vez vamos a utilizar capas de QKeras.
QKeras es "Quantized Keras" para la cuantificación heterogénea profunda de modelos de ML.

https://github.com/google/qkeras

Está mantenido por Google y recientemente agregamos soporte para modelos de QKeras en hls4ml.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from callbacks import all_callbacks
from tensorflow.keras.layers import Activation
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu

Estamos utilizando la capa `QDense` en lugar de `Dense`, y `QActivation` en lugar de `Activation`. También estamos especificando `kernel_quantizer = quantized_bits(6,0,0)`. Esto utilizará 6 bits (de los cuales 0 son enteros) para los pesos. También usamos la misma cuantificación para los sesgos, y `quantized_relu(6)` para activaciones ReLU de 6 bits.

In [ ]:
model = Sequential()
model.add(
    QDense(
        64,
        input_shape=(16,),
        name='fc1',
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
model.add(QActivation(activation=quantized_relu(6), name='relu1'))
model.add(
    QDense(
        32,
        name='fc2',
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
model.add(QActivation(activation=quantized_relu(6), name='relu2'))
model.add(
    QDense(
        32,
        name='fc3',
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
model.add(QActivation(activation=quantized_relu(6), name='relu3'))
model.add(
    QDense(
        5,
        name='output',
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
model.add(Activation(activation='softmax', name='softmax'))

## Entrenar con esparcidad
Volvamos a entrenar con esparcidad en el modelo, ya que las capas de QKeras son podables.

In [ ]:
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

pruning_params = {"pruning_schedule": pruning_schedule.ConstantSparsity(0.75, begin_step=2000, frequency=100)}
model = prune.prune_low_magnitude(model, **pruning_params)

## Entrenar el modelo
Utilizaremos la misma configuración que el modelo de la parte 1: optimizador Adam con pérdida categórica de entropía cruzada.
Los callbacks disminuirán la tasa de aprendizaje y guardarán el modelo en un directorio llamado 'model_2'.
El modelo no es muy complejo, por lo que esto debería tomar solo unos minutos incluso en la CPU.
Si has reiniciado el kernel del cuaderno después de entrenar una vez, establece `train = False` para cargar el modelo entrenado en lugar de entrenar nuevamente.

In [ ]:
train = True
if train:
    adam = Adam(lr=0.0001)
    model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
    callbacks = all_callbacks(
        stop_patience=1000,
        lr_factor=0.5,
        lr_patience=10,
        lr_epsilon=0.000001,
        lr_cooldown=2,
        lr_minimum=0.0000001,
        outputDir='model_3',
    )
    callbacks.callbacks.append(pruning_callbacks.UpdatePruningStep())
    model.fit(
        X_train_val,
        y_train_val,
        batch_size=1024,
        epochs=30,
        validation_split=0.25,
        shuffle=True,
        callbacks=callbacks.callbacks,
    )
    # Save the model again but with the pruning 'stripped' to use the regular layer types
    model = strip_pruning(model)
    model.save('model_3/KERAS_check_best_model.h5')
else:
    from tensorflow.keras.models import load_model
    from qkeras.utils import _add_supported_quantized_objects

    co = {}
    _add_supported_quantized_objects(co)
    model = load_model('model_3/KERAS_check_best_model.h5', custom_objects=co)

## Comprobar rendimiento
¿Cómo se compara este modelo, que fue entrenado usando 6 bits y 75% de esparcidad, con el modelo original? Informemos sobre la precisión y hagamos una curva ROC. El modelo cuantificado y podado se muestra con líneas sólidas, mientras que el modelo no podado de la parte 1 se muestra con líneas discontinuas.

También debemos verificar que hls4ml pueda respetar la elección de usar 6 bits en todo el modelo y que coincida con la precisión. Generaremos una configuración a partir de este modelo cuantificado, y graficaremos su rendimiento como la línea punteada.
La configuración generada se imprime. Notarás que utiliza 7 bits para el tipo, ¿pero especificamos 6? Eso se debe a que QKeras no cuenta el bit de signo cuando especificamos el número de bits, por lo que el tipo que realmente se utiliza necesita 1 más.

También utilizaremos el pase de optimización `OutputRoundingSaturationMode` de `hls4ml` para establecer las capas de activación en redondear, en lugar de truncar, el fundido. Esto es importante para obtener una buena precisión del modelo cuando se utilizan activaciones de precisión de bits pequeños. Y estableceremos un tipo de datos diferente para las tablas utilizadas en el Softmax, solo para un poco de rendimiento adicional.

**Asegúrate de haber entrenado el modelo de la parte 1**

In [ ]:
import hls4ml
import plotting

config = hls4ml.utils.config_from_keras_model(model, granularity='name')
config['LayerName']['softmax']['exp_table_t'] = 'ap_fixed<18,8>'
config['LayerName']['softmax']['inv_table_t'] = 'ap_fixed<18,4>'
print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=config, output_dir='model_3/hls4ml_prj', part='xcu250-figd2104-2L-e'
)
hls_model.compile()

y_qkeras = model.predict(np.ascontiguousarray(X_test))
y_hls = hls_model.predict(np.ascontiguousarray(X_test))
np.save('model_3/y_qkeras.npy', y_qkeras)
np.save('model_3/y_hls.npy', y_hls)

In [ ]:
%matplotlib inline
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model

model_ref = load_model('model_1/KERAS_check_best_model.h5')
y_ref = model_ref.predict(X_test)

print("Accuracy baseline:  {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_ref, axis=1))))
print("Accuracy pruned, quantized: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_qkeras, axis=1))))
print("Accuracy hls4ml: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_hls, axis=1))))

fig, ax = plt.subplots(figsize=(9, 9))
_ = plotting.makeRoc(y_test, y_ref, classes)
plt.gca().set_prop_cycle(None)  # reset the colors
_ = plotting.makeRoc(y_test, y_qkeras, classes, linestyle='--')
plt.gca().set_prop_cycle(None)  # reset the colors
_ = plotting.makeRoc(y_test, y_hls, classes, linestyle=':')

from matplotlib.lines import Line2D

lines = [Line2D([0], [0], ls='-'), Line2D([0], [0], ls='--'), Line2D([0], [0], ls=':')]
from matplotlib.legend import Legend

leg = Legend(ax, lines, labels=['baseline', 'pruned, quantized', 'hls4ml'], loc='lower right', frameon=False)
ax.add_artist(leg)

## Sintetizar
Ahora vamos a sintetizar este modelo cuantificado y podado.

**La síntesis llevará un tiempo**

Mientras se ejecuta la síntesis en C, podemos monitorear el progreso observando el archivo de registro abriendo un terminal desde el inicio del cuaderno y ejecutando:

`tail -f model_3/hls4ml_prj/vivado_hls.log`

In [ ]:
hls_model.build(csim=False)

## Verificar los informes
Imprime los informes generados por Vivado HLS. Presta especial atención a la sección "Estimaciones de utilización" esta vez.

In [ ]:
hls4ml.report.read_vivado_report('model_3/hls4ml_prj')

Imprime el informe del modelo entrenado en la parte 1. Ahora, en comparación con el modelo de la parte 1, este modelo ha sido entrenado con cuantificación de baja precisión y un 75% de podado. Deberías poder ver que hemos ahorrado muchos recursos en comparación con donde comenzamos en la parte 1. Al mismo tiempo, consultando la curva ROC anterior, ¡el rendimiento del modelo es prácticamente idéntico incluso con esta compresión drástica!

**Nota que necesitas haber entrenado y sintetizado el modelo de la parte 1**

In [ ]:
hls4ml.report.read_vivado_report('model_1/hls4ml_prj')

Imprime el informe del modelo entrenado en la parte 3. Ambos modelos fueron entrenados con un 75% de esparcidad, pero el nuevo modelo también utiliza una precisión de 6 bits. Puedes ver cómo Vivado HLS ha trasladado las operaciones de multiplicación de DSP a LUTs, reduciendo el uso de recursos "críticos".

**Nota que necesitas haber entrenado y sintetizado el modelo de la parte 3**

In [ ]:
hls4ml.report.read_vivado_report('model_2/hls4ml_prj')

## Nota
Ten en cuenta también que las estimaciones de recursos de Vivado HLS tienden a _sobreestimar_ las LUTs, mientras que generalmente estiman correctamente los DSPs. Ejecutar las etapas posteriores de compilación de FPGA revela el uso de recursos más realista. Puedes ejecutar el siguiente paso, 'síntesis lógica', con `hls_model.build(synth=True, vsynth=True)`, pero lo hemos omitido en este tutorial por razones de tiempo.